# 更少的内存

---

注：本课的理解需要其他知识作为补充，请查看文件：PQ算法.md; <br/>

In [35]:
from operator import index

import faiss
import numpy as np
import time

# 固定随机种子
np.random.seed(42)

# 创建全局数据与查询向量
dim = 256
data = np.random.rand(100000, dim)
query = np.random.rand(2, dim)

In [36]:
# 构造装饰器计算程序运行时间

def time_calculator(test_function):

    # 定义包装函数
    def wrapper():
        print('*' * 13)
        print("程序运行开始！")
        start_time = time.time()
        test_function()
        print(f"程序运行时间是:{time.time() - start_time}")
        print('*' * 13, end='\n' * 2)

    return wrapper


In [37]:
# 使用最普通的暴力搜索方法
@time_calculator
def test01():
    index = faiss.IndexFlatL2(dim)
    index.add(data)
    start_time_test = time.time()
    D, I = index.search(query, k=2)
    print(f"查询所需时间为:{time.time()-start_time_test:.4f}s")
    print(f"最短距离为：{D}")
    print(f"最短距离索引为：{I}")
    faiss.write_index(index, "IndexFlatL2.faiss")


# 使用常规的IndexIVFFlat进行存储，并查看内存使用
@time_calculator
def test02():
    quantizer = faiss.IndexFlatL2(dim)
    index = faiss.IndexIVFFlat(quantizer, dim, 500)  # 量化器、维度、聚类中心
    assert not index.is_trained
    index.train(data)
    assert index.is_trained
    index.add(data)
    start_time_test = time.time()
    index.nprobe = 1
    D, I = index.search(query, k=2)
    print(f"查询所需时间为:{time.time()-start_time_test:.4f}s")
    print(f"最短距离为：{D}")
    print(f"最短距离索引为：{I}")
    faiss.write_index(index, "IndexIVFFlat.faiss")


# 使用IndexIVFPQ进行存储，并查看内存使用
'''参数1：量化器
参数2：维度
参数3：聚类中心的个数
参数4：划分子向量空间的个数
参数5：子向量空间内的聚类中心(位)'''
@time_calculator
def test03():
    quantizer = faiss.IndexFlatL2(dim)
    index = faiss.IndexIVFPQ(quantizer, dim, 500, 8, 8) # 2^8 = 256
    assert not index.is_trained
    index.train(data)
    assert index.is_trained
    index.add(data)
    start_time_test = time.time()
    index.nprobe = 1
    D, I = index.search(query, k=2)
    print(f"查询所需时间为:{time.time()-start_time_test:.4f}s")
    print(f"最短距离为：{D}")
    print(f"最短距离索引为：{I}")
    faiss.write_index(index, "IndexIVFIP.faiss")

In [39]:
# 执行程序
if __name__ == "__main__":
    total_start_time = time.time()
    print("IndexFlatL2:")
    test01()
    print("IndexIVFFlat:")
    test02()
    print("IndexIVFPQ:", end='\n' * 2)
    test03()
    print(f"程序总运行时间是:{time.time() - total_start_time}")

IndexFlatL2:
*************
程序运行开始！
查询所需时间为:0.0074s
最短距离为：[[30.664503 30.716288]
 [31.105942 31.285503]]
最短距离索引为：[[25371 25699]
 [61674 73419]]
程序运行时间是:0.2448439598083496
*************

IndexIVFFlat:
*************
程序运行开始！
查询所需时间为:0.0000s
最短距离为：[[31.998238 32.928993]
 [32.32107  34.518463]]
最短距离索引为：[[13923   418]
 [ 3146  9136]]
程序运行时间是:1.3011128902435303
*************

IndexIVFPQ:

*************
程序运行开始！
查询所需时间为:0.0000s
最短距离为：[[21.978174 22.520166]
 [22.428432 23.09396 ]]
最短距离索引为：[[39077 11784]
 [29962 58354]]
程序运行时间是:5.635815382003784
*************

程序总运行时间是:7.181772232055664


<img src="./Code_Result.png" width="460">

我们可以发现，暴力搜索搜索时间仍然是最久的，但是结果最优秀的。在IndexIVFPQ搜索中看似出现了**更小的距离**，但实际上是因为编码和解码的**误差**导致的，并不是**真实距离**

### 评价内存消耗情况
既然是**更少的内存**，那么实际上内存的消耗情况是怎么样的呢？

<img src="./memory.png" width="706">

从图中的储存占用可以看到，IndexIVFFlat方法占用的储存最多（原始数据加上倒排表结构），IndexIVFPQ方法占用的储存最小